In [ ]:
# krx 전종목 기본정보에 대한 기존 data가 아직 존재하는지에 대한 점검 코드이다.


async def get_finaldata(self):


    try:
      new =  await self.get_issued_all()      # 크롤링하는 함수를 호출하여 , 새로운 data를 받아오는 과정
      original = await self.get_origin_data()  # 기존 db의 data를 가져오기 위한 함수를 호출
 
    
      for o in original:
          found= False.   # 이 found 변수를 이용하는 것이 정말 중요했다.
        
          for n in new:
            # 기존 db에 있는 키값이 새로운 db에도 살아있다는 의미이다. 이때는 그 키값애 대한 data가 바꼈을수도 있으니 새로운 data로 update를 해주자 
            if o[1] == n['ticker_symbol']:
              await self.db.execute(f"UPDATE listed_issue SET ticker_name_full = '{n['ticker_name_full']}', ticker_name = '{n['ticker_name']}' ,  market_type =  '{n['market_type']}' , share_type = '{n['share_type']} ', listing_date ='{n['listing_date']}' , listed_share = '{n['listed_share']}' , par_value = '{n['par_value']}' where ticker_symbol  ='{o[1]}'")
              await self.db.commit()
              print(o[1])
              print('update')
              found = True
              break
          


          # 기존 db에 존재했던 키 값이 이제는 존재하지 않을떄를 의미 -> 이떄는 종목이 삭제됨을 의미하므로 상폐처리를 해준다 .
          if not found:   
            print(f'o[1] : {o[1]}')
            await self.db.execute(f"UPDATE listed_issue SET maintain = False where ticker_symbol ='{o[1]}'")
            await self.db.commit()
            print("fail")